# Import librairies

In [11]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import numpy as np
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.mixture import GaussianMixture

# Import data

In [2]:
path = "../data"
df = pd.read_csv(os.path.join(path,"cleaned_dataset.csv"),index_col=0)

In [102]:
df = df[df["basesalary"]<=2*df["basesalary"].mean()]

# Add gaussian noise by columns

In [105]:
noises_std={}
for col in df.columns:
    noises_std[col] = df[col].std()    

In [106]:
df2 = df.copy()

In [107]:
for col in df2.columns:
    df2[col]=df2[col]+np.random.normal(0,noises_std[col],len(df))

In [108]:
X = df2.drop(columns=["basesalary"])
y = df2["basesalary"]

In [109]:
regressor = RandomForestRegressor(n_estimators = 10, random_state = 0, max_depth=10)
regressor.fit(X, y) 
y_pred = regressor.predict(X)
rmse = float(format(np.sqrt(mean_squared_error(y, y_pred)), '.3f'))
print("\nRMSE: ", rmse)


RMSE:  47122.597


In [110]:
y_pred = regressor.predict(df.drop(columns=["basesalary"]))
rmse = float(format(np.sqrt(mean_squared_error(df["basesalary"], y_pred)), '.3f'))
print("\nRMSE: ", rmse)


RMSE:  28458.664


In [111]:
pd.concat([df2["yearsofexperience"],df["yearsofexperience"]],axis=1).corr()

,yearsofexperience,yearsofexperience
yearsofexperience,1.000000,0.708437
yearsofexperience,0.708437,1.000000


# Add gaussian noise by rows

# Gaussian mixture

In [113]:
gm = GaussianMixture(n_components=20, random_state=0,covariance_type='diag').fit(df)

In [114]:
df_sample = gm.sample(50000)[0]
y_gaussian = df_sample[:,3]
x_gaussian = df_sample[:,[i for i in range(len(df.columns)) if i!=3]]
regressor = RandomForestRegressor(n_estimators = 30, random_state = 0, max_depth=10)
regressor.fit(x_gaussian, y_gaussian) 

RandomForestRegressor(max_depth=10, n_estimators=30, random_state=0)

In [115]:
rmse = float(format(np.sqrt(mean_squared_error(y, y_pred)), '.3f'))
print("\nRMSE: ", rmse)


RMSE:  48668.581


In [116]:
y_pred = regressor.predict(df.drop(columns=["basesalary"]))
rmse = float(format(np.sqrt(mean_squared_error(df["basesalary"], y_pred)), '.3f'))
print("\nRMSE: ", rmse)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(



RMSE:  160890.363


In [117]:
df["basesalary"].mean()

155892.7930736961